# Предобработка данных для рекомендательной системы

In [1]:
import pandas as pd


class Constants:
    USER_ID = "user_id"
    ITEM_ID = "item_id"
    TIMESTAMP = "time"

    TRANSACTIONS_PATH = "/Users/alfa/Documents/diplom/graphnn-recommendation-system/data/transactions_train.csv"
    RESULT_PATH = "/Users/alfa/Documents/diplom/graphnn-recommendation-system/data/processed_transactions_train.csv"

In [2]:
def prepare_filtered_dataset(
    transactions: pd.DataFrame,
    num_customers: int = 1000,
    min_articles_per_user: int = 5,
    min_users_per_article: int = 5,
) -> pd.DataFrame:
    """
    Отбираем сбалансированный набор данных:
    - num_customers: количество пользователей для отбора
    - min_articles_per_user: минимальное количество покупок у пользователя
    - min_users_per_article: минимальное количество покупателей у товара
    """

    # Первичная фильтрация пользователей
    user_purchase_counts = transactions[Constants.USER_ID].value_counts()
    active_users = user_purchase_counts[user_purchase_counts >= min_articles_per_user]

    # Отбираем топ-N самых активных пользователей
    selected_users = active_users.head(num_customers).index

    # Фильтруем транзакции только по выбранным пользователям
    user_filtered = transactions[transactions[Constants.USER_ID].isin(selected_users)]

    # Фильтрация товаров
    article_popularity = user_filtered[Constants.ITEM_ID].value_counts()
    popular_articles = article_popularity[
        article_popularity >= min_users_per_article
    ].index

    # Окончательный набор данных
    final_data = user_filtered[user_filtered[Constants.ITEM_ID].isin(popular_articles)]

    print(f"Исходный размер данных: {len(transactions)} транзакций")
    print(f"Отобрано пользователей: {len(selected_users)}")
    print(f"Отобрано товаров: {len(popular_articles)}")
    print(f"Финальный размер данных: {len(final_data)} транзакций")

    return final_data.reset_index(drop=True)

In [3]:
transactions = pd.read_csv(Constants.TRANSACTIONS_PATH)
transactions = transactions.rename(
    columns={
        "t_dat": Constants.TIMESTAMP,
        "customer_id": Constants.USER_ID,
        "article_id": Constants.ITEM_ID,
    }
)
transactions = prepare_filtered_dataset(transactions=transactions)
transactions.to_parquet(Constants.RESULT_PATH, index=False)

Исходный размер данных: 31788324 транзакций
Отобрано пользователей: 1000
Отобрано товаров: 29998
Финальный размер данных: 485117 транзакций


In [4]:
transactions.head()

,time,user_id,item_id,price,sales_channel_id
0,2018-09-20,01e464bf74b13a55df22de1528eff2b33749c0cd92953b...,519583008,0.030492,2
1,2018-09-20,01e464bf74b13a55df22de1528eff2b33749c0cd92953b...,539878003,0.042356,2
2,2018-09-20,01e464bf74b13a55df22de1528eff2b33749c0cd92953b...,448515018,0.038119,2
3,2018-09-20,01e464bf74b13a55df22de1528eff2b33749c0cd92953b...,448515001,0.038119,2
4,2018-09-20,02703e242da346fc85f1aad21208d1103e62e705564c81...,628813002,0.013542,2
